<a href="https://colab.research.google.com/github/azganushpoghosyan/statistics/blob/master/hypothesis_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import numpy as np
import scipy.stats as ss
from IPython.display import Math, Latex
from IPython.core.display import Image

# Introduction
Hypothesis testing is a statistical method crucial in drawing inferences about population parameters from sample data. This notebook explores the key components of hypothesis testing and demonstrates how to apply it using functions from the scipy library in Python.

#More on Theory

**Hypothesis testing** is a statistical method used to draw inferences about a population parameter based on a sample of data. Several key components play a role in conducting hypothesis testing. A **statistical model** defines our observations, and at its core, hypothesis testing calculates the probability, under this model, of observing an outcome equal to or more extreme than what we observed. The statistical model, along with its assumptions, is crucial for accurate probability calculation.

The **null hypothesis** ${H_0}$ represents the status quo, acting as a default assumption about the target parameter, while the **alternative hypothesis** ${H_1}$contradicts the null hypothesis, suggesting a specific change or effect. To illustrate these concepts, consider a paid product with a certain conversion rate (r). We introduce a new feature and claim that it will siginificantly increase the conversion rate. In this case the experimental setups will involve two independent random samples from the same user base, assigned to control and test groups. The null hypothesis is that the conversion rates are equal: r_0 = r_1, and the alternative hypothesis is that the new rate is higher than the default value r1 > r0. The associated statistical model is a Bernoulli model, assuming independent user actions.

The outcomes of hypothesis testing are either **rejecting the null hypothesis**, indicating that the observed data make the null hypothesis unlikely, or **failing to reject the null hypothesis** due to insufficient evidence. These outcomes are asymmetric, dependent on observed data. If the null hypothesis is rejected, the experiment is deemed significant.

Decisions on rejecting the null hypothesis hinge on the **test statistic** (T), a function of random variables modeling the data. The choice of a suitable test statistic, distinguishing between null and alternative hypotheses, is crucial.  For example, in our case, the test statistic could be the count of converted users in the test group, assuming a binomial distribution under the null hypothesis.


The decision to reject the null hypothesis relies on two crucial factors: the **significance level (α)** and the **p-value**. The significance level signifies the likelihood of making a Type I error (false positive) and is predetermined at the beginning of the experiment. Meanwhile, the p-value assesses how well the observed data align with the null hypothesis—a smaller p-value indicates data less consistent with the null hypothesis.

An understanding of errors in hypothesis testing is foundational. A **Type I error** transpires when the null hypothesis is true, yet we erroneously reject it. Conversely, a **Type II error** occurs when the null hypothesis is false, but we fail to reject it. While the significance level is typically set at 0.05, it can vary depending on experimental constraints.

Furthermore, the **power of the test**, indicating the probability of accurately rejecting the null hypothesis when the alternative hypothesis is true, complements the probability of committing a Type II error.

Finally, two main types of tests exist: **two-tailed** and **one-tailed**. Two-tailed tests imply a departure from the null hypothesis without specifying a higher or lower parameter. One-tailed tests claim a higher or lower parameter, indicating an increasing or decreasing effect compared to the ground truth. The choice depends on the nature of the research question and the expected direction of the effect.

Now, let's implement a hypothesis test using our product example. Suppose we have 30,000 users in each of the test and control groups. They use the product for a specific time, and at the end of this period, we observe 9,000 users converted from the control group and 9,300 users converted from the test group, which experienced the new feature. We've ensured that there are no other differences between the test and control groups except for the new feature in the product. The conversion rate in the control group is our ground truth, set at 0.3, and we will calculate the probability of observing the data in the test group based on the Bernoulli statistical model with a rate of 0.3.

In this case, the test statistic is the sum of converted users, which follows a binomial distribution, equal to 9,300 in the test group. We'll set the significance level of the test to be α = 0.05. The main question is how probable this value, or a more extreme value, is under the null hypothesis. This is a one-sided test, as we assert that the conversion will be higher in the test group. In statistical terms, we aim to calculate the probability of observing 9,300 or a larger number of successes in a binomial distribution with a success rate of 0.3 and a total number of trials of 30,000. More formally, we need to solve the following equation for x when α = 0.05:

P(binomial(30000, 0.3) > x) = α

which is equal to:

P(binomial(30000, 0.3) < x) = 1 - α

We can calculate this using **scipy.stats.binom** object's **ppf** method

In [27]:
# The ppf (percent point function) method calculates the inverse of the cumulative distribution function (cdf)
test_stat_threshold = ss.binom.ppf(0.95, 30000, 0.3)
print(test_stat_threshold)

9131.0


We've established that for x = 9131 or larger, we can reject the null hypothesis at a significance level α = 0.05. In our case, the count of converted users is 9300.

Now, let's calculate the p-value. As mentioned earlier, the p-value is the probability of observing a test statistic as extreme or more extreme than the value observed under the null hypothesis. In our scenario, it's P(T > 9300), where T is the test statistic representing the number of converted users. This statistic follows a binomial distribution with parameters p = 0.3 and n = 30000. Using the **scipy.stats.binom** object, we can calculate this probability: P(binom(30000, 0.3) > 9300), which is equivalent to 1 - P(binom(30000, 0.3) < 9300).

So, for x = 9131 or larger we can reject the null hypothesis at a significance level α = 0.05. In our case it's much larger than 9131, we got the total number of converted users to be equal 9300.
Now let's calculate the p-value in our experiment. As mentioned earlier p-value is the probability of observing a test statistics as extreme or more extreme than the value observed under the null hypothesis. so, it's P(T > 9300) where T is the test statistics, that is the number of converted users which follows a binomial distribution with p = 0.3 and n = 30000. Again, using the binom  object we can calculate this probability:

P(binomial(30000, 0.3) > 9300)

which is the same as

1 - P(binomial(30000, 0.3) < 9300)

In [ ]:
# We'll use the sf (survival function) method, defined as 1 - cdf, to calculate the p-value
p_value = ss.binom.sf(9300, 30000, 0.3)
print('{:.20f}'.format(p_value))

Since the p-value is significantly smaller than the chosen alpha level (0.05), we have strong evidence to reject the null hypothesis.

Now, let's explore a different scenario within the same product context. Suppose we're interested in the time spent per user, claiming that the new feature will alter this metric. Unlike before, we don't assert whether it will increase or decrease; our claim is simply that it will change. This calls for a two-sided test.

Assuming the time spent per user follows a normal distribution based on our existing userbase data, we conduct a hypothesis test. The setup is identical to the previous case: the new version to the test group, the original version to the control group.

The null hypothesis posits that the time spent per user is the same in both groups, while the alternative hypothesis suggests a difference. Our test statistic is the average time spent per user in the two groups.

To reject the null hypothesis, we aim to show that the probability of observing a test statistic with an absolute value greater than some threshold is less than our chosen significance level, alpha (set at 0.05).

Applying a t-test, suitable for normal distributions and allowing estimation of variance from observations without knowledge of the population variance, involves calculating the test statistic as follows:

$$
\begin{equation}
 T = \frac{\bar{X_1} - \bar{X_2}}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
\end{equation}
$$

Here, $\bar{X_1}$ and $\bar{X_2}$ are sample means, ${s_1^2}$ and ${s_2^2}$ are sample standard deviations, and ${n_1}$ and ${n_2}$ are sample sizes. We use the unbiased sample standard deviation (ddof=1) when calculating standard deviations.

Now, let's assume we have calculated the target statistic for both samples, each with 100 observations:

$\bar{X_1}$ = 25
$\bar{X_2}$ = 30
${s_1^2}$ = 5
${s_2^2}$ = 7
${n_1}$ = ${n_2}$ = 100

We can calculate the test statistic, degrees of freedom, and determine the probability of observing a test statistic as extreme or more extreme than the one in the test group. Alternatively, we can utilize the scipy libraryto streamline this process.


In [28]:
# We can use the scipy library's ttest_ind_from_stats function to calculate the test statistic and p-value for a two-sided t-test.
statistic, p_value = ss.ttest_ind_from_stats(mean1=25, std1=5, nobs1=100,
                                             mean2=30, std2=7, nobs2=100, equal_var=True, alternative='two-sided')

# Output the results
print("Test Statistic: ", statistic)
print("P-value: {:.20f}".format(p_value))

Test Statistic:  -5.812381937190964
P-value: 0.00000002432003215120


Apparently, at a significance level of 0.05, we can reject the null hypothesis in favor of the alternative one. This implies that the average time spent per user in the alternative version is significantly different from the control version.

This example serves to demonstrate the use of two-sided tests. It's worth noting that, from a business perspective, a one-sided t-test could have been more favorable. Such a test would specifically assess whether the new feature increases the time spent per user rather than just detecting a difference.


#Conclusion
This exploration of hypothesis testing delved into its foundational concepts, methodologies, and practical applications. From establishing null and alternative hypotheses to navigating significance levels, p-values, and test statistics, we examined the intricacies of both one-sided and two-sided tests. The illustrative examples involving conversion rates and time spent per user underscored the real-world relevance of hypothesis testing in making informed decisions. As a vital component of statistical analysis, hypothesis testing empowers researchers and analysts to draw meaningful conclusions about population parameters based on sample data, ensuring robust and evidence-driven insights.